### Imports

In [9]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import spacy
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, log_loss
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FunctionTransformer
from spacy.lang.es.stop_words import STOP_WORDS
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder

In [7]:
# Cargar datos
df = pd.read_csv('/content/drive/MyDrive/NLP/TP/Airline_Reviews.csv')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Preprocesamiento y limpieza

In [12]:
python -m spacy download es_core_news_sm
import spacy
from spacy.lang.es.stop_words import STOP_WORDS

SyntaxError: invalid syntax (<ipython-input-12-bd10379a4e19>, line 1)

In [11]:
# Preprocesamiento

# Cargar el modelo de lenguaje español de spaCy
nlp = spacy.load('es_core_news_sm')

# Lista de stopwords en español
stopwords = STOP_WORDS

# Función de preprocesamiento
def preprocess(text, remove_stopwords=True):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if token.is_alpha]
    if remove_stopwords:
        tokens = [token for token in tokens if token not in stopwords]
    return ' '.join(tokens)

# Aplicar preprocesamiento a la columna de reseñas
df['cleaned_review'] = df['Review'].apply(lambda x: preprocess(x, remove_stopwords=True))

# Comprobar y eliminar filas con textos vacíos después del preprocesamiento
df = df[df['cleaned_review'].str.strip().astype(bool)]

OSError: [E050] Can't find model 'es_core_news_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
# Análisis de Sentimiento con TextBlob
def get_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity

df['sentiment'] = df['cleaned_review'].apply(get_sentiment)
df['sentiment_class'] = pd.cut(df['sentiment'], bins=[-1, -0.05, 0.05, 1], labels=['Malo', 'Promedio', 'Bueno'])

# Eliminar filas con valores nulos en 'sentiment_class'
df = df.dropna(subset=['sentiment_class'])

In [ ]:
# # Análisis Exploratorio y Visualizaciones
# positive_reviews = ' '.join(df[df['sentiment_class'] == 'Bueno']['cleaned_review'])
# negative_reviews = ' '.join(df[df['sentiment_class'] == 'Malo']['cleaned_review'])

# wordcloud = WordCloud(width=800, height=400).generate(positive_reviews)
# plt.figure(figsize=(10, 5))
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis('off')
# plt.show()

# wordcloud = WordCloud(width=800, height=400).generate(negative_reviews)
# plt.figure(figsize=(10, 5))
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis('off')
# plt.show()

### Sentiment Analysis

#### BoW + Regresión Logística

In [ ]:
# Asegurarse de que las etiquetas categóricas estén en formato categórico
df['sentiment_class'] = df['sentiment_class'].astype('category')
df['sentiment_class_code'] = df['sentiment_class'].cat.codes

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df['cleaned_review'], df['sentiment_class_code'], test_size=0.2, random_state=42)

# Convertir series de pandas a arrays de numpy (para asegurar que no sean de solo lectura)
X_train = np.array(X_train.copy())
X_test = np.array(X_test.copy())
y_train = np.array(y_train, dtype=np.int32, copy=True)
y_test = np.array(y_test, dtype=np.int32, copy=True)

pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', LogisticRegression())
])

# Definir los parámetros
parameters = {
    'vect__max_features': [5000, 10000, 20000],
    'vect__ngram_range': [(1, 1), (1, 2)],
    'clf__C': [0.1, 1, 10]
}

# Realizar la búsqueda en malla
grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Imprimir los mejores parámetros y el mejor puntaje
print(grid_search.best_params_)
print(grid_search.best_score_)

# Evaluación del mejor modelo
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
y_prob = best_model.predict_proba(X_test)

# Evaluación usando entropía cruzada
cross_entropy = log_loss(y_test, y_prob)
print(f"Cross Entropy Loss: {cross_entropy}")

# Evaluación usando métricas estándar
print(classification_report(y_test, y_pred))

C:\Users\Usuario\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


{'clf__C': 0.1, 'vect__max_features': 20000, 'vect__ngram_range': (1, 1)}
0.7718498141481118
Cross Entropy Loss: 0.5261437336256444
              precision    recall  f1-score   support

           0       0.77      0.80      0.78      1313
           1       0.54      0.41      0.47       980
           2       0.85      0.92      0.88      2333

    accuracy                           0.78      4626
   macro avg       0.72      0.71      0.71      4626
weighted avg       0.76      0.78      0.77      4626



#### TF-IDF + Regresión Logística

In [ ]:
# Asegurarse de que las etiquetas categóricas estén en formato categórico
df['sentiment_class'] = df['sentiment_class'].astype('category')

# Convertir etiquetas categóricas a códigos numéricos
df['sentiment_class_code'] = df['sentiment_class'].cat.codes

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df['cleaned_review'], df['sentiment_class_code'], test_size=0.2, random_state=42)

# Convertir series de pandas a arrays de numpy (para asegurar que no sean de solo lectura)
X_train = np.array(X_train.copy())
X_test = np.array(X_test.copy())
y_train = np.array(y_train, dtype=np.int32, copy=True)
y_test = np.array(y_test, dtype=np.int32, copy=True)

# Definir el pipeline con preprocesamiento
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression())
])

# Parámetros para GridSearchCV
parameters = {
    'tfidf__max_features': [5000, 10000, 20000],
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'clf__C': [0.1, 1, 10]
}

# Realizar GridSearchCV
grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Imprimir los mejores parámetros y la mejor puntuación
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Score: {grid_search.best_score_}")

# Evaluación del mejor modelo
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
y_prob = best_model.predict_proba(X_test)

# Evaluación usando entropía cruzada
cross_entropy = log_loss(y_test, y_prob)
print(f"Cross Entropy Loss: {cross_entropy}")

# Evaluación usando métricas estándar
print(classification_report(y_test, y_pred))

C:\Users\Usuario\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Best Parameters: {'clf__C': 10, 'tfidf__max_features': 20000, 'tfidf__ngram_range': (1, 1)}
Best Score: 0.7629313333868859
Cross Entropy Loss: 0.5264325366470037
              precision    recall  f1-score   support

           0       0.76      0.77      0.77      1313
           1       0.50      0.43      0.47       980
           2       0.86      0.90      0.88      2333

    accuracy                           0.77      4626
   macro avg       0.71      0.70      0.70      4626
weighted avg       0.76      0.77      0.76      4626



#### Red Neuronal Recurrente (RNN)

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels
from tensorflow.keras.models import clone_model

def create_model(num_layers=1, units=128, dropout_rate=0.2, learning_rate=0.001, frozen_layers=0):
    model = Sequential()
    model.add(Embedding(input_dim=20000, output_dim=128, input_length=100))
    for _ in range(num_layers):
        model.add(LSTM(units=units, dropout=dropout_rate, recurrent_dropout=dropout_rate, trainable=(not frozen_layers)))
    model.add(Dense(units=3, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

class KerasClassifierWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, model):
        self.model = model
        self.classes_ = None

    def fit(self, X, y):
        X, y = check_X_y(X, y)
        self.classes_ = unique_labels(y)
        self.model.fit(X, y, epochs=5, batch_size=32, verbose=0)
        return self

    def predict(self, X):
        check_is_fitted(self)
        X = check_array(X)
        return self.model.predict_classes(X)

    def predict_proba(self, X):
        check_is_fitted(self)
        X = check_array(X)
        return self.model.predict_proba(X)

# Utiliza la función create_model para crear tu modelo
model = create_model()

# Envuelve tu modelo de Keras en el wrapper personalizado
wrapped_model = KerasClassifierWrapper(model)

# Define el parámetro grid
param_grid = {
    'num_layers': [1, 2],
    'units': [64, 128],
    'dropout_rate': [0.2, 0.3],
    'learning_rate': [0.001, 0.0001],
    'epochs': [5, 10],
    'batch_size': [32, 64],
    'frozen_layers': [0, 1]
}

# Realiza la búsqueda de cuadrícula
grid = GridSearchCV(estimator=wrapped_model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1)
grid_result = grid.fit(X_train, y_train)

# Imprime los mejores resultados
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 128 candidates, totalling 384 fits


C:\Users\Usuario\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


ValueError: Cannot have number of splits n_splits=3 greater than the number of samples: n_samples=2.

In [ ]:
from sklearn.model_selection import GridSearchCV, LeaveOneOut
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam
import numpy as np

# Función para crear el modelo
def create_model(num_layers=1, units=128, dropout_rate=0.2, learning_rate=0.001, frozen_layers=0):
    model = Sequential()
    model.add(Embedding(input_dim=20000, output_dim=128, input_length=100))
    for _ in range(num_layers):
        model.add(LSTM(units=units, dropout=dropout_rate, recurrent_dropout=dropout_rate, trainable=(not frozen_layers)))
    model.add(Dense(units=3, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Wrap the Keras model so it can be used by scikit-learn
model = create_model()

from sklearn.model_selection import LeaveOneOut
import itertools

# Define the parameter grid
param_grid = {
    'num_layers': [1, 2],
    'units': [64, 128],
    'dropout_rate': [0.2, 0.3],
    'learning_rate': [0.001, 0.0001],
    'epochs': [5, 10],
    'batch_size': [32, 64],
    'frozen_layers': [0, 1]
}

# Create a LeaveOneOut object
loo = LeaveOneOut()

best_score = 0
best_params = None

# Iterate over all parameter combinations
for params in itertools.product(*param_grid.values()):
    num_layers, units, dropout_rate, learning_rate, epochs, batch_size, frozen_layers = params
    model = create_model(num_layers=num_layers, units=units, dropout_rate=dropout_rate, learning_rate=learning_rate, frozen_layers=frozen_layers)
    scores = []

    # Perform Leave-One-Out Cross Validation
    for train_index, test_index in loo.split(X_train):
        X_train_fold, X_test_fold = X_train[train_index], X_train[test_index]
        y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]

        model.fit(X_train_fold, y_train_fold, epochs=epochs, batch_size=batch_size, verbose=0)
        _, accuracy = model.evaluate(X_test_fold, y_test_fold, verbose=0)
        scores.append(accuracy)

    # Compute average score
    avg_score = np.mean(scores)

    # Update best score and best params if current score is better
    if avg_score > best_score:
        best_score = avg_score
        best_params = params

# Print best score and best parameters
print("Best score:", best_score)
print("Best params:", best_params)


KeyError: "None of [Index([0], dtype='int32')] are in the [index]"

#### Modelo Basado en Transformador (BERT)

In [ ]:
# Importar las bibliotecas necesarias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from datasets import Dataset

# Configuración del dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Dividir los datos en conjuntos de entrenamiento y prueba
train_texts, test_texts, train_labels, test_labels = train_test_split(df['cleaned_review'], df['sentiment_class'], test_size=0.2, random_state=42)

# Cargar el tokenizador y el modelo preentrenado de BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=3).to(device)

default_label = len(label_map)  # Asigna un valor por defecto al último valor del mapeo
label_map = {
    "Bueno": 0,
    "Promedio": 1,
    "Malo": 2,
}

train_labels_mapped = [label_map Fa.get(label, default_label) for label in train_labels.tolist()]
# Crear el Dataset de Hugging
test_labels_mapped = [label_map.get(label, default_label) for label in test_labels.tolist()]
ce con las etiquetas mapeadas
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels_mapped
})

test_dataset = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask'],
    'labels': test_labels_mapped
})


# Definir los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=0.5,  # Reducir el número de épocas
    per_device_train_batch_size=2,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=15000,  # Guardar el modelo menos frecuentemente
)

# Crear el DataCollator para manejar el padding dinámico
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Crear el Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# Entrenar el modelo
trainer.train()

# Evaluar el modelo
predictions = trainer.predict(test_dataset)
preds = torch.argmax(predictions.predictions, dim=1)
accuracy = accuracy_score(test_labels, preds)
report = classification_report(test_labels, preds)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  2%|▏         | 163/9251 [32:07<29:50:48, 11.82s/it]
                                                   
  0%|          | 10/4626 [01:15<9:44:42,  7.60s/it]

{'loss': 1.1783, 'grad_norm': 8.972036361694336, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.0}


                                                   
  0%|          | 20/4626 [02:32<9:46:24,  7.64s/it]

{'loss': 1.178, 'grad_norm': 273.867919921875, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.0}


                                                   
  1%|          | 30/4626 [03:48<9:39:12,  7.56s/it]

...

                                                      
100%|██████████| 4626/4626 [17:35:47<00:00, 13.69s/it]


{'train_runtime': 63347.474, 'train_samples_per_second': 0.146, 'train_steps_per_second': 0.073, 'train_loss': 1.0525455187266903, 'epoch': 0.5}


100%|██████████| 579/579 [1:14:27<00:00,  7.72s/it]


TypeError: argmax(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

 32%|███▏      | 75/232 [19:39:16<41:08:37, 943.42s/it]
